# 1. Prediksi Penjualan Produk Paragon

Pada project kali ini saya akan melakukan peramalan penjualan minggu selanjutnya produk dari Paragon. Data yang dimiliki merupakan data timeseries yang memiliki rentang antara minggu ke 52 tahun 2021 sampai dengan minggu ke 14 tahun 2023.

Saya akan melakukan prediksi penjualan dengan mengguakan Random Forest Regression.

# 2. Introduction

Nama saya Stefanus Bayu  Waskito dan saya merupakan siswa Hacktiv8 Batch Remote 18.

# 3. External Link

#4. Working Area

## 4.1 Import Library

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

## 4.2 Import Data

In [2]:
# Data Loading

data = pd.read_csv('https://raw.githubusercontent.com/ardhiraka/talent_fair_sample_challenge/main/datasets/sample_dataset_timeseries_noarea.csv')
data

,week_number,week_start_date,week_end_date,product_item,quantity
0,2021-52,2021-12-27,2022-01-02,0073377087c7ddace313ff829cd7b0b3,41
1,2021-52,2021-12-27,2022-01-02,012d2bb142b2d5e57330acbfd8e9ac52,430
2,2021-52,2021-12-27,2022-01-02,0192926e33d1153b63511a5b83eca843,4
3,2021-52,2021-12-27,2022-01-02,019502f6d13e383caa9b9673bf1302f8,148
4,2021-52,2021-12-27,2022-01-02,01e7ca6d4e21badc1d4225b6b2c7bd9e,119
...,...,...,...,...,...
102728,2023-14,2023-04-03,2023-04-09,ff3890faa80ff425ae906e27bbc50bc6,907
102729,2023-14,2023-04-03,2023-04-09,ff4b0f3b9c78c91510a4e3920fbc5090,81
102730,2023-14,2023-04-03,2023-04-09,ff9d05a4d6f922b3ab1653f58165b8ce,16
102731,2023-14,2023-04-03,2023-04-09,ffbd6f1d884056a4f622d54ae79e5a14,42920


## 4.3 Feature Engineering

In [3]:
# Tampilkan Shape Data

data.shape

(102733, 5)

Data yang dimiliki terdiri dari 102733 baris data dan 5 fitur data.

In [4]:
# Tampilkan jumlah baris data yang terduplikasi

duplikasi = data.duplicated().sum()
print('Total baris data yang terduplikasi sebanyak', duplikasi)

Total baris data yang terduplikasi sebanyak 0


In [5]:
# Cek Missing Value

data.isna().sum()

week_number        0
week_start_date    0
week_end_date      0
product_item       2
quantity           0
dtype: int64

In [6]:
# Tampilkan baris yang memiliki missing value

baris_missing_value = data[data['product_item'].isnull()]
print('Baris data yang memiliki missing value \n')
baris_missing_value

Baris data yang memiliki missing value 



,week_number,week_start_date,week_end_date,product_item,quantity
18048,2022-12,2022-03-21,2022-03-27,NaN,119
19536,2022-13,2022-03-28,2022-04-03,NaN,147


In [7]:
# Hapus Missing Value Pada product_item

data.dropna(subset=['product_item'], inplace=True)

In [8]:
# Cek setelah missing value dihapus

data.isna().sum()

week_number        0
week_start_date    0
week_end_date      0
product_item       0
quantity           0
dtype: int64

In [9]:
# Buat fungsi untuk mengubah nama produk menjadi produk 1 dst untuk mempermudah dapal pengolahan data

def rename(data, product_item, prefix='produk'):

    # membuat mapping nama kategori lama ke baru
    mapping = {old_name: f'{prefix} {i+1}' for i, old_name in enumerate(data[product_item].unique())}

    # melakukan penggantian nama kategori pada kolom yang diberikan
    data[product_item] = data[product_item].replace(mapping)

    return data

Membuat fungsi untuk melakukan perubahan nama pada fitur produk item dengan tujuan mempermudah analisis.

In [10]:
# Ubah nama produk

data = rename(data, 'product_item')

Merubah nama produk dalam fitur product_item

In [11]:
# Tampilkan sampel data setelah nama produk dihapus

data.head(10)

,week_number,week_start_date,week_end_date,product_item,quantity
0,2021-52,2021-12-27,2022-01-02,produk 1,41
1,2021-52,2021-12-27,2022-01-02,produk 2,430
2,2021-52,2021-12-27,2022-01-02,produk 3,4
3,2021-52,2021-12-27,2022-01-02,produk 4,148
4,2021-52,2021-12-27,2022-01-02,produk 5,119
5,2021-52,2021-12-27,2022-01-02,produk 6,2
6,2021-52,2021-12-27,2022-01-02,produk 7,44
7,2021-52,2021-12-27,2022-01-02,produk 8,165
8,2021-52,2021-12-27,2022-01-02,produk 9,41
9,2021-52,2021-12-27,2022-01-02,produk 10,383


In [12]:
# Menghitung total unique value pada fitur product_item

unik_produk = data['product_item'].nunique()
print('Total unique value pada fitur product_item adalah', unik_produk)

Total unique value pada fitur product_item adalah 2309


In [13]:
# Drop fitur yang tidak digunakan

data.drop(['week_start_date', 'week_end_date'], axis=1, inplace=True)

Saya melakukan penghapusan fitur 'week_start_date' dan 'week_end_date' karena sudah diwakili dengan fitur week_number

In [14]:
# Tampilkan head data setelah beberapa fitur dihapus

data.head()

,week_number,product_item,quantity
0,2021-52,produk 1,41
1,2021-52,produk 2,430
2,2021-52,produk 3,4
3,2021-52,produk 4,148
4,2021-52,produk 5,119


In [15]:
# Lakukan split data

split_point = '2022-52'
train = data[data['week_number'] < split_point].copy()
val = data[data['week_number'] >= split_point].copy()

Melakukan split data menjadi data train dan data val dengan split poin di 'week_number' : '2022-52'

In [16]:
# Tampilkan head data train

train.head()

,week_number,product_item,quantity
0,2021-52,produk 1,41
1,2021-52,produk 2,430
2,2021-52,produk 3,4
3,2021-52,produk 4,148
4,2021-52,produk 5,119


In [17]:
# Tampilkan head data val atau test

val.head()

,week_number,product_item,quantity
79942,2022-52,produk 452,27
79943,2022-52,produk 1,26
79944,2022-52,produk 453,2838
79945,2022-52,produk 1752,1418
79946,2022-52,produk 455,49


In [18]:
# Tambahkan kolom baru dengan nama 'quantity_next_week'

train['quantity_next_week'] = train.groupby("product_item")['quantity'].shift(-1)
val['quantity_next_week'] = val.groupby("product_item")['quantity'].shift(-1)

In [19]:
# Hapus missing value pada data train

train = train.dropna()

In [20]:
# Menambah kolom 'lag_quantity' pada data train

train["lag_quantity"] = train.groupby("product_item")['quantity'].shift(1)
val["lag_quantity"] = val.groupby("product_item")['quantity'].shift(1)


In [21]:
# Menambah kolom diff_quantity pada data train dan val

train["diff_quantity"] = train.groupby("product_item")['quantity'].diff(1)
val["diff_quantity"] = val.groupby("product_item")['quantity'].diff(1)

In [22]:
# Tampilkan data train dengan produk_item = produk 1 setelah membuat kolom lag_quantity da val_quantity

train[train['product_item'] == 'produk 1'].head()

,week_number,product_item,quantity,quantity_next_week,lag_quantity,diff_quantity
0,2021-52,produk 1,41,461.0,NaN,NaN
452,2022-01,produk 1,461,486.0,41.0,420.0
1953,2022-02,produk 1,486,406.0,461.0,25.0
3430,2022-03,produk 1,406,452.0,486.0,-80.0
4910,2022-04,produk 1,452,368.0,406.0,46.0


In [23]:
# tambah kolom mean_quantity pada data val untuk menyimpan nilai rolling quantitiy

val["mean_quantity"] = val.groupby("product_item")['quantity'].shift(1)

In [24]:
# Hitung rolling quantity

train.groupby("product_item")['quantity'].rolling(4).mean()

product_item       
produk 1      0           NaN
              452         NaN
              1953        NaN
              3430     348.50
              4910     451.25
                        ...  
produk 999    71142    576.75
              72749    650.25
              74357    722.50
              75956    770.00
              77553    716.75
Name: quantity, Length: 77725, dtype: float64

In [25]:
train.groupby("product_item")['quantity'].rolling(4).mean().reset_index(level=0, drop=True)

0           NaN
452         NaN
1953        NaN
3430     348.50
4910     451.25
          ...  
71142    576.75
72749    650.25
74357    722.50
75956    770.00
77553    716.75
Name: quantity, Length: 77725, dtype: float64

In [26]:
# Tambahkan nilai rolling mean_quantity ke datafraem 

train["mean_quantity"] = train.groupby("product_item")['quantity'].rolling(4).mean().reset_index(level=0, drop=True)
train[train['product_item'] == 'produk 1'].head()

,week_number,product_item,quantity,quantity_next_week,lag_quantity,diff_quantity,mean_quantity
0,2021-52,produk 1,41,461.0,NaN,NaN,NaN
452,2022-01,produk 1,461,486.0,41.0,420.0,NaN
1953,2022-02,produk 1,486,406.0,461.0,25.0,NaN
3430,2022-03,produk 1,406,452.0,486.0,-80.0,348.50
4910,2022-04,produk 1,452,368.0,406.0,46.0,451.25


In [27]:
# Metric evaluasi dengan menggunakan mape dan weighted mape

def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    ape[~np.isfinite(ape)] = 1. # pessimist estimate
    return np.mean(ape)

def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

In [28]:
# Hitung nilai mape pada train

y_pred = train['quantity']
y_true = train['quantity_next_week']

mape(y_true, y_pred)

0.6322776380293141

Dari evaluasi menggunakna mape dapat dilihat nilai mape adalah 0.63 atau nilai error sebesar 63%

In [29]:
# Hitung nilai weighted mape pada train

wmape(y_true, y_pred)

0.26964181042658053

Dari evaluasi menggunakan weighted mape dapat dilihat nilai weighted mape adalah 0.269

## 4.4 Modeling

In [30]:
# Model Menggunakan RandomForrrestRegressor

features = ['quantity', 'lag_quantity', 'diff_quantity', 'mean_quantity']
imputer = SimpleImputer()
Xtr = imputer.fit_transform(train[features])
ytr = train['quantity_next_week']


model = RandomForestRegressor(n_estimators=25, random_state=0, n_jobs=6)
model.fit(Xtr, ytr)

RandomForestRegressor(n_estimators=25, n_jobs=6, random_state=0)

Mendefinidikan model dengan menggunakan random forest regressor yang hyperparameternya terdiri dari n_estimator sebesar 25, tidak memberi nilai pada random state karena data yang dimiliki timeseries dan n_job = 6

In [31]:
Xval = imputer.transform(val[features])
yval = val['quantity_next_week']

predicted = model.predict(Xval)

In [32]:
# Evaluasi Model

print('mape =', mape(yval, predicted))
print('weigted mape =', wmape(yval, predicted))

mape = 3.430375542140661
weigted mape = 0.3047407397396759


In [33]:
train['quantity_next_two_week'] = train.groupby("product_item")['quantity'].shift(-2)
val['quantity_next_two_week'] = val.groupby("product_item")['quantity'].shift(-2)

In [34]:
# Tampilkan Prediksi

train[train['product_item'] == 'produk 1'].head()

,week_number,product_item,quantity,quantity_next_week,lag_quantity,diff_quantity,mean_quantity,quantity_next_two_week
0,2021-52,produk 1,41,461.0,NaN,NaN,NaN,486.0
452,2022-01,produk 1,461,486.0,41.0,420.0,NaN,406.0
1953,2022-02,produk 1,486,406.0,461.0,25.0,NaN,452.0
3430,2022-03,produk 1,406,452.0,486.0,-80.0,348.50,368.0
4910,2022-04,produk 1,452,368.0,406.0,46.0,451.25,423.0


## 4.5 Model Inference

In [35]:
train = train.dropna(subset=['quantity_next_week', 'quantity_next_two_week'])
imputer = SimpleImputer()
Xtr = imputer.fit_transform(train[features])
ytr = train[['quantity_next_week', 'quantity_next_two_week']]

mdl = RandomForestRegressor(n_estimators=25, random_state=0, n_jobs=6)
mdl.fit(Xtr, ytr)

RandomForestRegressor(n_estimators=25, n_jobs=6, random_state=0)

In [36]:
Xval = imputer.transform(val[features])
yval = val[['quantity_next_week', 'quantity_next_two_week']]

p = mdl.predict(Xval)


In [37]:
mape(yval, p)

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


quantity_next_week        3.771719
quantity_next_two_week    5.083476
dtype: float64

In [38]:
wmape(yval, p)

quantity_next_week        0.309213
quantity_next_two_week    0.342729
dtype: float64

In [39]:
tes = val[(val['week_number'] == '2023-14') & (val['product_item'] == 'produk 1')]
tes.head()

,week_number,product_item,quantity,quantity_next_week,lag_quantity,diff_quantity,mean_quantity,quantity_next_two_week
101233,2023-14,produk 1,4,NaN,5.0,-1.0,5.0,NaN


In [40]:
p = mdl.predict(tes[features])
p

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


array([[ 3.49933333, 15.72133333]])

In [41]:
tes['predicted_sales_next_week'] = p[:, 0]
tes['predicted_sales_next_two_week'] = p[:, 1]

<ipython-input-41-7681c11603c4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tes['predicted_sales_next_week'] = p[:, 0]
<ipython-input-41-7681c11603c4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tes['predicted_sales_next_two_week'] = p[:, 1]


In [42]:
# Menampilkan prediksi Minggu selanjutnya dan 2 minggu selanjutnya

tes.head()

,week_number,product_item,quantity,quantity_next_week,lag_quantity,diff_quantity,mean_quantity,quantity_next_two_week,predicted_sales_next_week,predicted_sales_next_two_week
101233,2023-14,produk 1,4,NaN,5.0,-1.0,5.0,NaN,3.499333,15.721333


## 5. Conclusion

Dari data yang dimiliki, terdapat 2 missing value pada data yaitu pada fitur product_item. Penghapusan kolom yang memiliki missing value dilakukan karena menurut saya tidak dapat dilakukan inmputasi karena juka dilakuka imputasi dapat mengakibatkan bias.

Penerapan RandomForestRegression dengan memberi nilai n_job = 6 dan n_estimator = 25. Pada Training memiliki nilai MAPE sebesar 0.632 dan nilai WMAPE = 0.269 sedangkan pada Testing memilii MAPE sebesar 3.34 dan WMAPE sebesar 0.304.

Model dapat memprediksi kuantitas penjualan minggu selanjutnya dan 2 minggu selanjutnya. Dilakukan uji coba prediksi untuk produk 1 pada week_number 2023-14 atau minggu terakhir, didapatkan hasil prediksi penjualan minggu selanjutnya sebanyak 3 produk dan 2 minggu setelahnya 16 produk.